# Verify a FHIR Digitally Signing FHIR Bundle or QuestionnaireResponse Object

This is a Jupyter Notebook using Python 3.7 and openSSl to create JSON Web Signature (JWS)(see RFC 7515) and attach it to a FHIR Bundle or QuestionnaireResponse resource.

- If the resource is a Bundle use Bundle.signature
- If the resource is a QuestionnaireResponse use its [Signature extension](http://hl7.org/fhir/StructureDefinition/questionnaireresponse-signature
)

See: http://build.fhir.org/signatures.html

*Although self-signed certificates are used for the purpose of these examples, they are not recommended for production systems.*

### Import Libraries

In [1]:
from requests import get, post
from json import dumps, loads
from pathlib import Path
# from datetime import datetime
# import pytz
from jose import jws  #python JWS package
from base64 import urlsafe_b64encode, b64decode
from jcs import canonicalize #package for a JCS (RFC 8785) compliant canonicalizer.

/Users/ehaas/.pyenv/versions/fhir_builds/lib/python3.7/site-packages/jose/backends/cryptography_backend.py:4: CryptographyDeprecationWarning: Python 3.7 is no longer supported by the Python core team and support for it is deprecated in cryptography. The next release of cryptography will remove support for Python 3.7.
  from cryptography.exceptions import InvalidSignature, InvalidTag


In [ ]:
local_file = True

# bundle_id = "f50e72a6-290e-44f8-9b6f-adf19713d0d4/_history/183"  # insert bundle or questionnaireresponse id here
bundle_id = 'Bundle-cdex-searchbundle-digital-sig-example.json'
bundle_id = 'Bundle-cdex-searchbundle-digital-sig-example.json'
bundle_id = 'Bundle-cdex-searchbundle-digital-sig-example.json'

def fetch_payload():

  # url = "https://argopatientlist.aidbox.app/fhir/Bundle"
  # url = 'http://test.fhir.org/r4/Bundle'
  url = 'https://hl7.org/fhir/us/davinci-cdex'

  username = "basic"
  password = "secret"
  headers = {"Accept": "application/fhir+json" , "Content-Type": "application/fhir+json"}

  r = get(f'{url}/{bundle_id}', auth=(username, password), headers = headers)
  my_obj = r.json()
  # print("="*80)
  # print("STATUS: ",r.status_code)

  # print("="*80)
  # print("HEADERS:\n")
  # for k,v in r.headers.items():
  #     print(f'{k} = {v}')
  # print("="*80)
  # print("BODY:\n")
  # print(dumps(my_obj,indent=2))
  return(my_obj)

if local_file: 
  # in_file = 'in_files/object_to_sign.json'
  in_path = '/Users/ehaas/Documents/FHIR/davinci-ecdx/output'
  path = Path() in_path / bundle_id
  my_obj =loads(path.read_text())
else:
  my_obj = fetch_payload()
print(dumps(my_obj,indent=2))

{
  "resourceType": "Bundle",
  "id": "cdex-searchbundle-digital-sig-example",
  "type": "searchset",
  "total": 1,
  "link": [
    {
      "relation": "self",
      "url": "http://hapi.fhir.org/baseR4/Condition?patient=06e1f0dd-5fbe-4480-9bb4-6b54ec02d31b"
    }
  ],
  "entry": [
    {
      "fullUrl": "http://hapi.fhir.org/baseR4/Condition/4ac41715-fcbd-421c-8796-9b2c9706dd3f",
      "resource": {
        "resourceType": "Condition",
        "id": "4ac41715-fcbd-421c-8796-9b2c9706dd3f",
        "meta": {
          "versionId": "10",
          "lastUpdated": "2020-04-28T20:28:00.008+00:00"
        },
        "text": {
          "status": "generated",
          "div": "<div xmlns=\"http://www.w3.org/1999/xhtml\"><a name=\"Condition_4ac41715-fcbd-421c-8796-9b2c9706dd3f\"> </a><p class=\"res-header-id\"><b>Generated Narrative: Condition 4ac41715-fcbd-421c-8796-9b2c9706dd3f</b></p><a name=\"4ac41715-fcbd-421c-8796-9b2c9706dd3f\"> </a><a name=\"hc4ac41715-fcbd-421c-8796-9b2c9706dd3f\"> </a

## Receiver/Verifier Steps

###  1. Remove the Signature from the Bundle or QR

In [3]:
def  pop_element(resource, element):
  try:
    my_element = resource.pop(element) # remove element
    return my_element 
  except KeyError:
    pass
 
if my_obj['resourceType'] == 'Bundle':
  recd_signature = pop_element(my_obj, 'signature') #for Bundles
elif my_obj['resourceType'] == 'QuestionnaireResponse':
  try:
    for i, extension in enumerate(my_obj['extension']):
       if extension['url'] == 'http://hl7.org/fhir/StructureDefinition/questionnaireresponse-signature':
          my_obj_signature_ext = my_obj['extension'].pop(i) # remove element
          recd_signature = my_obj_signature_ext['valueSignature']
    if i == 0:
      my_obj.pop('extension') # remove extension if empty
  except KeyError:
    print('No signature extension found')
else:
  print('Not a Bundle or QuestionnaireResponse')

recd_signature

{'type': [{'system': 'urn:iso-astm:E1762-95:2013',
   'code': '1.2.840.10065.1.12.1.5',
   'display': 'Verification Signature'}],
 'when': '2020-10-23T04:54:56.048+00:00',
 'who': {'identifier': {'type': {'coding': [{'system': 'http://terminology.hl7.org/CodeSystem/v2-0203',
      'code': 'NPI'}]},
   'system': 'http://hl7.org/fhir/sid/us-npi',
   'value': '9941339100'},
  'display': 'John Hancock, MD'},
 'onBehalfOf': {'identifier': {'system': 'http://hl7.org/fhir/sid/us-npi',
   'value': '1184932014'}},
 'targetFormat': 'application/fhir+json;canonicalization=http://hl7.org/fhir/canonicalization/json#document',
 'sigFormat': 'application/jose',
 'data': 'ZXlKaGJHY2lPaUpTVXpJMU5pSXNJbXRwWkNJNklqZGpNall3Tm1JMk5UQm1OemczTUdNMk1HUmtPRGhtTkRNMFptVmtZV0ZtWkRCbU0yWTNNemtpTENKcmRIa2lPaUpTVXlJc0luTnBaMVFpT2lJeU1ESXdMVEV3TFRJelZEQTBPalUwT2pVMkxqQTBPQ3N3TURvd01DSXNJblI1Y0NJNklrcFhWQ0lzSW5nMVl5STZXeUpOU1VsR1YycERRMEU0UzJkQmQwbENRV2RKVlVwRlRqSXhUME14UlRSMmJWRlhMMDFUTjJoMk9ITnZWMEpuYzNkRVVWbEtTMjl

### 2. Canonicalize the resource using IETF JSON Canonicalization Scheme (JCS):

- Remove the id and meta elements if present before canonicalization

In [4]:
my_obj_id = pop_element(my_obj, 'id')
my_obj_meta = pop_element(my_obj, 'meta')
canonical_obj = canonicalize(my_obj)
canonical_obj, len(canonical_obj)

(b'{"entry":[{"fullUrl":"http://hapi.fhir.org/baseR4/Condition/4ac41715-fcbd-421c-8796-9b2c9706dd3f","resource":{"category":[{"coding":[{"code":"encounter-diagnosis","display":"Encounter Diagnosis","system":"http://terminology.hl7.org/CodeSystem/condition-category"}]}],"clinicalStatus":{"coding":[{"code":"active","system":"http://terminology.hl7.org/CodeSystem/condition-clinical"}]},"code":{"coding":[{"code":"122481008","display":"Hammer toe (disorder)","system":"http://snomed.info/sct"}],"text":"Hammer Toe"},"encounter":{"reference":"http://example.org/cdex/provider/fhir/Encounter/5fe62cd5-bfcf-4d3b-a1e9-80d6f75d6f82"},"id":"4ac41715-fcbd-421c-8796-9b2c9706dd3f","meta":{"lastUpdated":"2020-04-28T20:28:00.008+00:00","versionId":"10"},"onsetDateTime":"2018-10-21T21:22:15-07:00","recordedDate":"2018-10-21T21:22:15-07:00","resourceType":"Condition","subject":{"reference":"http://example.org/cdex/provider/fhir/Patient/06e1f0dd-5fbe-4480-9bb4-6b54ec02d31b"},"text":{"div":"<div xmlns=\\"http

### 3. Transform canonicalize Bundle to a base64 format using the Base64-URL algorithm.

In [5]:
recd_b64_canonical_obj  = urlsafe_b64encode(canonical_obj).decode()
recd_b64_canonical_obj = recd_b64_canonical_obj.replace("=","")  #remove padding since decode package doesn't use them 
recd_b64_canonical_obj

'eyJlbnRyeSI6W3siZnVsbFVybCI6Imh0dHA6Ly9oYXBpLmZoaXIub3JnL2Jhc2VSNC9Db25kaXRpb24vNGFjNDE3MTUtZmNiZC00MjFjLTg3OTYtOWIyYzk3MDZkZDNmIiwicmVzb3VyY2UiOnsiY2F0ZWdvcnkiOlt7ImNvZGluZyI6W3siY29kZSI6ImVuY291bnRlci1kaWFnbm9zaXMiLCJkaXNwbGF5IjoiRW5jb3VudGVyIERpYWdub3NpcyIsInN5c3RlbSI6Imh0dHA6Ly90ZXJtaW5vbG9neS5obDcub3JnL0NvZGVTeXN0ZW0vY29uZGl0aW9uLWNhdGVnb3J5In1dfV0sImNsaW5pY2FsU3RhdHVzIjp7ImNvZGluZyI6W3siY29kZSI6ImFjdGl2ZSIsInN5c3RlbSI6Imh0dHA6Ly90ZXJtaW5vbG9neS5obDcub3JnL0NvZGVTeXN0ZW0vY29uZGl0aW9uLWNsaW5pY2FsIn1dfSwiY29kZSI6eyJjb2RpbmciOlt7ImNvZGUiOiIxMjI0ODEwMDgiLCJkaXNwbGF5IjoiSGFtbWVyIHRvZSAoZGlzb3JkZXIpIiwic3lzdGVtIjoiaHR0cDovL3Nub21lZC5pbmZvL3NjdCJ9XSwidGV4dCI6IkhhbW1lciBUb2UifSwiZW5jb3VudGVyIjp7InJlZmVyZW5jZSI6Imh0dHA6Ly9leGFtcGxlLm9yZy9jZGV4L3Byb3ZpZGVyL2ZoaXIvRW5jb3VudGVyLzVmZTYyY2Q1LWJmY2YtNGQzYi1hMWU5LTgwZDZmNzVkNmY4MiJ9LCJpZCI6IjRhYzQxNzE1LWZjYmQtNDIxYy04Nzk2LTliMmM5NzA2ZGQzZiIsIm1ldGEiOnsibGFzdFVwZGF0ZWQiOiIyMDIwLTA0LTI4VDIwOjI4OjAwLjAwOCswMDowMCIsInZlcnNpb25JZCI6IjEwIn0sIm9uc2V0RGF

### 4. Get the base64 encoded JWS  from the `Bundle.signature.data`  element

In [6]:
recd_b64_jws = recd_signature['data']
recd_b64_jws

'ZXlKaGJHY2lPaUpTVXpJMU5pSXNJbXRwWkNJNklqZGpNall3Tm1JMk5UQm1OemczTUdNMk1HUmtPRGhtTkRNMFptVmtZV0ZtWkRCbU0yWTNNemtpTENKcmRIa2lPaUpTVXlJc0luTnBaMVFpT2lJeU1ESXdMVEV3TFRJelZEQTBPalUwT2pVMkxqQTBPQ3N3TURvd01DSXNJblI1Y0NJNklrcFhWQ0lzSW5nMVl5STZXeUpOU1VsR1YycERRMEU0UzJkQmQwbENRV2RKVlVwRlRqSXhUME14UlRSMmJWRlhMMDFUTjJoMk9ITnZWMEpuYzNkRVVWbEtTMjlhU1doMlkwNUJVVVZNUWxGQmQyZGFWWGhEZWtGS1FtZE9Wa0pCV1ZSQmJGWlVUVkpOZDBWUldVUldVVkZKUkVGd1JGbFhlSEJhYlRsNVltMXNhRTFTU1hkRlFWbEVWbEZSU0VSQmJGUlpXRlo2V1ZkNGNHUkhPSGhJVkVGaVFtZE9Wa0pCYjAxR1JWWTBXVmN4ZDJKSFZXZFVNMHB1V1ZjMWNHVnRSakJoVnpsMVRWSnJkMFozV1VSV1VWRkVSRUpDUzJJeWFIVkpSV2hvWW0xT2Rsa3ljM05KUlRGRlRWTk5kMGxSV1VwTGIxcEphSFpqVGtGUmEwSkdhRkp4WVVkR2RWa3lPV3BoTUVKc1pVZEdkR05IZUd4TWJUbDVXbnBCWlVaM01IbE9WRUV5VFdwQmVVMXFSVEJPVkZaaFJuY3dlVTU2UVRKTlZFRjVUV3BGTUU1VVZtRk5TVWRXVFZGemQwTlJXVVJXVVZGSFJYZEtWbFY2UlZSTlFrVkhRVEZWUlVOQmQwdFJNa1p6WVZkYWRtTnROWEJaVkVWVFRVSkJSMEV4VlVWQ2QzZEtWVEpHTVdNeVJuTmhXRkoyVFZJd2QwZDNXVVJXVVZGTFJFSlNSbVZIUm5SalIzaHNTVVU1ZVZveVJuVmhXSEJvWkVkc2R

### 5. Base64 decode the encoded JWS

note the signature is displayed with the parts labeled and separated with line breaks for easier viewing

In [7]:
labels = ['header', 'payload', 'signature']
recd_jws = b64decode(recd_b64_jws.encode()).decode()
for i,j in enumerate(recd_jws.split('.')):
    print(f'{labels[i]}:')
    print(f'{j}')
    print()

header:
eyJhbGciOiJSUzI1NiIsImtpZCI6IjdjMjYwNmI2NTBmNzg3MGM2MGRkODhmNDM0ZmVkYWFmZDBmM2Y3MzkiLCJrdHkiOiJSUyIsInNpZ1QiOiIyMDIwLTEwLTIzVDA0OjU0OjU2LjA0OCswMDowMCIsInR5cCI6IkpXVCIsIng1YyI6WyJNSUlGV2pDQ0E4S2dBd0lCQWdJVUpFTjIxT0MxRTR2bVFXL01TN2h2OHNvV0Jnc3dEUVlKS29aSWh2Y05BUUVMQlFBd2daVXhDekFKQmdOVkJBWVRBbFZUTVJNd0VRWURWUVFJREFwRFlXeHBabTl5Ym1saE1SSXdFQVlEVlFRSERBbFRZWFZ6WVd4cGRHOHhIVEFiQmdOVkJBb01GRVY0WVcxd2JHVWdUM0puWVc1cGVtRjBhVzl1TVJrd0Z3WURWUVFEREJCS2IyaHVJRWhoYm1Odlkyc3NJRTFFTVNNd0lRWUpLb1pJaHZjTkFRa0JGaFJxYUdGdVkyOWphMEJsZUdGdGNHeGxMbTl5WnpBZUZ3MHlOVEEyTWpBeU1qRTBOVFZhRncweU56QTJNVEF5TWpFME5UVmFNSUdWTVFzd0NRWURWUVFHRXdKVlV6RVRNQkVHQTFVRUNBd0tRMkZzYVdadmNtNXBZVEVTTUJBR0ExVUVCd3dKVTJGMWMyRnNhWFJ2TVIwd0d3WURWUVFLREJSRmVHRnRjR3hsSUU5eVoyRnVhWHBoZEdsdmJqRVpNQmNHQTFVRUF3d1FTbTlvYmlCSVlXNWpiMk5yTENCTlJERWpNQ0VHQ1NxR1NJYjNEUUVKQVJZVWFtaGhibU52WTJ0QVpYaGhiWEJzWlM1dmNtY3dnZ0dpTUEwR0NTcUdTSWIzRFFFQkFRVUFBNElCandBd2dnR0tBb0lCZ1FDakg1a20rSSs5NnFOd0JCTHh0RVlxM1FDa1k4am5nRGJqcFB4RDBXRWxXWFdpY0I0Z3I4

### 6. reattach the payload - the base64 encoded Bundle or QR - into the JWS payload element. 

note the signature is displayed with the parts labeled and separated with line breaks for easier viewing

In [8]:
split_sig = recd_jws.split('.')
split_sig[1] = recd_b64_canonical_obj
recd_jws = '.'.join(split_sig)
for i,j in enumerate(recd_jws.split('.')):
    print(f'{labels[i]}:')
    print(f'{j}')
    print()

header:
eyJhbGciOiJSUzI1NiIsImtpZCI6IjdjMjYwNmI2NTBmNzg3MGM2MGRkODhmNDM0ZmVkYWFmZDBmM2Y3MzkiLCJrdHkiOiJSUyIsInNpZ1QiOiIyMDIwLTEwLTIzVDA0OjU0OjU2LjA0OCswMDowMCIsInR5cCI6IkpXVCIsIng1YyI6WyJNSUlGV2pDQ0E4S2dBd0lCQWdJVUpFTjIxT0MxRTR2bVFXL01TN2h2OHNvV0Jnc3dEUVlKS29aSWh2Y05BUUVMQlFBd2daVXhDekFKQmdOVkJBWVRBbFZUTVJNd0VRWURWUVFJREFwRFlXeHBabTl5Ym1saE1SSXdFQVlEVlFRSERBbFRZWFZ6WVd4cGRHOHhIVEFiQmdOVkJBb01GRVY0WVcxd2JHVWdUM0puWVc1cGVtRjBhVzl1TVJrd0Z3WURWUVFEREJCS2IyaHVJRWhoYm1Odlkyc3NJRTFFTVNNd0lRWUpLb1pJaHZjTkFRa0JGaFJxYUdGdVkyOWphMEJsZUdGdGNHeGxMbTl5WnpBZUZ3MHlOVEEyTWpBeU1qRTBOVFZhRncweU56QTJNVEF5TWpFME5UVmFNSUdWTVFzd0NRWURWUVFHRXdKVlV6RVRNQkVHQTFVRUNBd0tRMkZzYVdadmNtNXBZVEVTTUJBR0ExVUVCd3dKVTJGMWMyRnNhWFJ2TVIwd0d3WURWUVFLREJSRmVHRnRjR3hsSUU5eVoyRnVhWHBoZEdsdmJqRVpNQmNHQTFVRUF3d1FTbTlvYmlCSVlXNWpiMk5yTENCTlJERWpNQ0VHQ1NxR1NJYjNEUUVKQVJZVWFtaGhibU52WTJ0QVpYaGhiWEJzWlM1dmNtY3dnZ0dpTUEwR0NTcUdTSWIzRFFFQkFRVUFBNElCandBd2dnR0tBb0lCZ1FDakg1a20rSSs5NnFOd0JCTHh0RVlxM1FDa1k4am5nRGJqcFB4RDBXRWxXWFdpY0I0Z3I4

### 7. Obtain public Key from the first certificate in JWS header `"x5c"` key

- base64 decode the key value
- Verify Issuer, Validity Dates, Subject,and KeyUsage of certificate ( with the source )
- Get the “Subject Public Key Info” from the cert to verify the signature

In [9]:
recd_header = jws.get_unverified_header(recd_jws)
recd_header

{'alg': 'RS256',
 'kid': '7c2606b650f7870c60dd88f434fedaafd0f3f739',
 'kty': 'RS',
 'sigT': '2020-10-23T04:54:56.048+00:00',
 'typ': 'JWT',
 'x5c': ['MIIFWjCCA8KgAwIBAgIUJEN21OC1E4vmQW/MS7hv8soWBgswDQYJKoZIhvcNAQELBQAwgZUxCzAJBgNVBAYTAlVTMRMwEQYDVQQIDApDYWxpZm9ybmlhMRIwEAYDVQQHDAlTYXVzYWxpdG8xHTAbBgNVBAoMFEV4YW1wbGUgT3JnYW5pemF0aW9uMRkwFwYDVQQDDBBKb2huIEhhbmNvY2ssIE1EMSMwIQYJKoZIhvcNAQkBFhRqaGFuY29ja0BleGFtcGxlLm9yZzAeFw0yNTA2MjAyMjE0NTVaFw0yNzA2MTAyMjE0NTVaMIGVMQswCQYDVQQGEwJVUzETMBEGA1UECAwKQ2FsaWZvcm5pYTESMBAGA1UEBwwJU2F1c2FsaXRvMR0wGwYDVQQKDBRFeGFtcGxlIE9yZ2FuaXphdGlvbjEZMBcGA1UEAwwQSm9obiBIYW5jb2NrLCBNRDEjMCEGCSqGSIb3DQEJARYUamhhbmNvY2tAZXhhbXBsZS5vcmcwggGiMA0GCSqGSIb3DQEBAQUAA4IBjwAwggGKAoIBgQCjH5km+I+96qNwBBLxtEYq3QCkY8jngDbjpPxD0WElWXWicB4gr8e9pHa6AA3pQc4MpBiQvVnUs7DmfmjNcolvqizvsgWZCODQm4pCbADNaO/0cDVagHXBsDgdmAxFRxHPMneUadwLa0pokRhAWd1hyz0xXTbdES1VjhYgOEB098Uq2shIzvIfc4dIFTPSy8Et+EpQAZo3+D68NAfWF9n4QyEVg8TH5YyqPUaQoEUKFr2Zm4+KFjVW9Xw3siVuV0wV2dGcU2L9QgDoxOX812t+UoLXuXFOXf9sRR

### 8. Inspect the x509 certificate for the

- Issuer
- Subject
- Key Usage
- Validaty dates
- SAN

Define the location of the Document Signing Certificate for comparison to verify 

 - See the Jupyter file [Create_Cert.ipynb]() for how to generate your own self-signed certificate.

In [10]:
recd_cert = b64decode(recd_header['x5c'][0])
with open('recd_cert.der', 'wb') as f:
    f.write(recd_cert)
certificate_path = Path('example_org_cert') # update this to your folder
print(f'Source certificate for comparison is at ~/{certificate_path}')
!openssl x509 -in recd_cert.der -inform DER -text

Source certificate for comparison is at ~/example_org_cert
Certificate:
    Data:
        Version: 3 (0x2)
        Serial Number:
            24:43:76:d4:e0:b5:13:8b:e6:41:6f:cc:4b:b8:6f:f2:ca:16:06:0b
        Signature Algorithm: sha256WithRSAEncryption
        Issuer: C = US, ST = California, L = Sausalito, O = Example Organization, CN = "John Hancock, MD", emailAddress = jhancock@example.org
        Validity
            Not Before: Jun 20 22:14:55 2025 GMT
            Not After : Jun 10 22:14:55 2027 GMT
        Subject: C = US, ST = California, L = Sausalito, O = Example Organization, CN = "John Hancock, MD", emailAddress = jhancock@example.org
        Subject Public Key Info:
            Public Key Algorithm: rsaEncryption
                Public-Key: (3072 bit)
                Modulus:
                    00:a3:1f:99:26:f8:8f:bd:ea:a3:70:04:12:f1:b4:
                    46:2a:dd:00:a4:63:c8:e7:80:36:e3:a4:fc:43:d1:
                    61:25:59:75:a2:70:1e:20:af:c7:bd:a4:76:ba:00:


### 9. Verify Signature using the public key or the X.509 Certificate

Alternatively:
1. visit https://jwt.io.
1. At the top of the page, select "RS256" for the algorithm.
1. Paste the JWS value printed below into the “Encoded” field.
1. The plaintext JWT will be displayed in the “Decoded:Payload” field.
1. The X509 Cert above will appear in the "Verify Signature" box.
1. If verified, a “Signature Verified” message will appear  in the bottom left hand corner.

In [11]:
recd_jws

'eyJhbGciOiJSUzI1NiIsImtpZCI6IjdjMjYwNmI2NTBmNzg3MGM2MGRkODhmNDM0ZmVkYWFmZDBmM2Y3MzkiLCJrdHkiOiJSUyIsInNpZ1QiOiIyMDIwLTEwLTIzVDA0OjU0OjU2LjA0OCswMDowMCIsInR5cCI6IkpXVCIsIng1YyI6WyJNSUlGV2pDQ0E4S2dBd0lCQWdJVUpFTjIxT0MxRTR2bVFXL01TN2h2OHNvV0Jnc3dEUVlKS29aSWh2Y05BUUVMQlFBd2daVXhDekFKQmdOVkJBWVRBbFZUTVJNd0VRWURWUVFJREFwRFlXeHBabTl5Ym1saE1SSXdFQVlEVlFRSERBbFRZWFZ6WVd4cGRHOHhIVEFiQmdOVkJBb01GRVY0WVcxd2JHVWdUM0puWVc1cGVtRjBhVzl1TVJrd0Z3WURWUVFEREJCS2IyaHVJRWhoYm1Odlkyc3NJRTFFTVNNd0lRWUpLb1pJaHZjTkFRa0JGaFJxYUdGdVkyOWphMEJsZUdGdGNHeGxMbTl5WnpBZUZ3MHlOVEEyTWpBeU1qRTBOVFZhRncweU56QTJNVEF5TWpFME5UVmFNSUdWTVFzd0NRWURWUVFHRXdKVlV6RVRNQkVHQTFVRUNBd0tRMkZzYVdadmNtNXBZVEVTTUJBR0ExVUVCd3dKVTJGMWMyRnNhWFJ2TVIwd0d3WURWUVFLREJSRmVHRnRjR3hsSUU5eVoyRnVhWHBoZEdsdmJqRVpNQmNHQTFVRUF3d1FTbTlvYmlCSVlXNWpiMk5yTENCTlJERWpNQ0VHQ1NxR1NJYjNEUUVKQVJZVWFtaGhibU52WTJ0QVpYaGhiWEJzWlM1dmNtY3dnZ0dpTUEwR0NTcUdTSWIzRFFFQkFRVUFBNElCandBd2dnR0tBb0lCZ1FDakg1a20rSSs5NnFOd0JCTHh0RVlxM1FDa1k4am5nRGJqcFB4RDBXRWxXWFdpY0I0Z3I4ZTlwSGE

In [12]:
!openssl x509 -in recd_cert.der -inform DER -pubkey -noout > recd-public-key.pem
with open('recd-public-key.pem') as f:
    pem_public = f.read()
try:
    verify = jws.verify(recd_jws, pem_public , algorithms=['RS256'])
    print('#     #                                                ### ')
    print('#     #  ######  #####   #  ######  #  ######  #####   ### ')
    print('#     #  #       #    #  #  #       #  #       #    #  ### ')
    print('#     #  #####   #    #  #  #####   #  #####   #    #   #  ')
    print(' #   #   #       #####   #  #       #  #       #    #      ')
    print('  # #    #       #   #   #  #       #  #       #    #  ### ')
    print('   #     ######  #    #  #  #       #  ######  #####   ### ')
    print('                                                           ')

except Exception as e:
    print('#    #   ####   #####      #    #  ######  #####   #  ######  #  ######  #####       ###          #   ')
    print('##   #  #    #    #        #    #  #       #    #  #  #       #  #       #    #       #          #    ')
    print('# #  #  #    #    #        #    #  #####   #    #  #  #####   #  #####   #    #           #####  #    ')
    print('#  # #  #    #    #        #    #  #       #####   #  #       #  #       #    #       #          #    ')
    print('#   ##  #    #    #         #  #   #       #   #   #  #       #  #       #    #      ###          #   ')
    print('#    #   ####     #          ##    ######  #    #  #  #       #  ######  #####        #            ## ')
    print('                ')
    print(f"not verified: {e}")

#     #                                                ### 
#     #  ######  #####   #  ######  #  ######  #####   ### 
#     #  #       #    #  #  #       #  #       #    #  ### 
#     #  #####   #    #  #  #####   #  #####   #    #   #  
 #   #   #       #####   #  #       #  #       #    #      
  # #    #       #   #   #  #       #  #       #    #  ### 
   #     ######  #    #  #  #       #  ######  #####   ### 
                                                           
